In [1]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Model, model_from_json
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing import image
import matplotlib.pyplot as plt
from scipy.misc import toimage
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras import utils
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.keras.callbacks import TensorBoard

import os
import time
%matplotlib inline 

In [2]:

# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 256, 256
input_shape = (img_width, img_height, 3)
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 30
# Размер мини-выборки
batch_size = 10
# Количество изображений для обучения
# 70/13/17
nb_train_samples = 100#7024   #//17500//?
# Количество изображений для проверки
nb_validation_samples = 305 #100#436#1436 #3750//?
# Количество изображений для тестирования
nb_test_samples = 1436   #3750//?

# Convolution Neuron Network Test Model
**Model  

Layer 0: Input image of size 256x256
* Layer 1: Convolution with 96 filters, size
11x11, stride 4, padding 2
* Layer 2: Max-Pooling with a size 3x3 filter,
stride 2
* Layer 3: Convolution with 192 filters, size 5x5,
stride 1, padding 2
* Layer 4: Max-Pooling with a size 3x3 filter,
stride 2
* Layer 5: Convolution with 288 filters, size 3x3,
stride 1, padding 1
* Layer 6: Convolution with 288 filters, size 3x3,
stride 1, padding 1
* Layer 7: Convolution with 192 filters, size 3x3,
stride 1, padding 1
* Layer 8: Max-Pooling with a size 3x3, stride 2
* Layer 9: Fully Connected with 4096 neurons
* Layer 10: Fully Connected with 4096 neurons
* Layer 11: Fully Connected with 5 neurons
* Result: Non-normalized log softmax scores, 5
classes

* lr = 0.00000005 = ussed *
* 5e-1weight decay every 5 epochs = ussed * 
* L2 regularization strength of 7.5e-2 - ussed *

* Kaiming weight initialization - need to use *

In [3]:
#87/87 [==============================] - 161s 2s/step - loss: 1.6241 - acc: 0.4892 - val_loss: 1.5705 - val_acc: 0.5104
#40 epoch
# model = Sequential()
# model.add(Conv2D(96, (11, 11), input_shape=input_shape, strides=4))#L1 
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(3,3), strides=2))#L2

# model.add(Conv2D(192, (5, 5), strides=1))#L3
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(3, 3), strides=2))#L4

# model.add(Conv2D(288, (3, 3)))#L5
# model.add(Activation('relu'))
# model.add(Conv2D(288, (3, 3)))#L6
# model.add(Activation('relu'))
# model.add(Conv2D(192, (3, 3)))#L7
# model.add(Activation('relu'))

# model.add(MaxPooling2D(pool_size=(3, 3), strides=2))#L8

# model.add(Flatten())
# model.add(Dense(512))#L9
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# # model.add(Dense(4096))#L10
# # model.add(Activation('relu'))
# model.add(Dense(5, activation='softmax', kernel_regularizer=l2(0.075)))#L11
# model.summary()
# model.compile(loss='categorical_crossentropy',
# #               optimizer=adam,
#               optimizer=Adam(lr=1e-5), 
#               metrics=['accuracy'])

In [4]:
# give:

#Epoch 25/60
#87/87 [==============================] - 2049s 24s/step - loss: 0.8586 - acc: 0.9425 - val_loss: 1.0638 - val_acc: 0.8646
#all learning 2 days

vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
vgg16_net.trainable = False
vgg16_net.summary()

model = Sequential()

model.add(vgg16_net)

model.add(Flatten())
model.add(Dense(256))#L9
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax', kernel_regularizer=l2(0.075)))#L11
model.summary() 

model.compile(loss='categorical_crossentropy',
#               optimizer=adam,
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [5]:
# adam = Adam(lr=0.00000005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.5, amsgrad=False)

In [6]:
datagen = ImageDataGenerator(rescale=1./ 255)

In [7]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

In [8]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

print(next(train_generator)[1].dtype)
print(next(train_generator)[1].shape)

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: 'train'

In [ ]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

print( nb_validation_samples // batch_size)
print(next(val_generator)[1].dtype)
print(next(val_generator)[1].shape)

In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

print(next(test_generator)[1].dtype)
print(next(test_generator)[1].shape)

In [ ]:
callbacks = [TensorBoard(log_dir='tb_logs', batch_size=batch_size, histogram_freq=1, write_images=True)]

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=1,
    callbacks=callbacks)

#nb_validation_samples // batch_size

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

In [ ]:
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

In [ ]:
directory = time.time() 
directory = time.ctime(directory) 

directory = time.strftime("%d_%b_%Y_%H_%Mmin", time.gmtime())
os.makedirs(directory + "")
print(directory)

In [44]:
# Open the file
# save model info
with open(directory + '/model_summary.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    model.summary(print_fn=lambda x: fh.write(x + '\n'))

    
# save learning config -----    
fh = open(directory + '/options.txt','w')

fh.write("input_shape: " + str(input_shape) + "\n");
fh.write("epochs: " + str(epochs) + "\n");

fh.write("batch_size: " + str(batch_size) + "\n");
fh.write("nb_train_samples: " + str(nb_train_samples) + "\n");
fh.write("nb_validation_samples: " + str(nb_train_samples) + "\n");
fh.write("nb_test_samples: " + str(nb_train_samples) + "\n");
fh.write("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))
fh.close()

NameError: name 'scores' is not defined

In [ ]:
model_json = model.to_json()
json_file = open(directory + "/mnist_model.json", "w")
# Записываем архитектуру сети в файл
json_file.write(model_json)
json_file.close()
# Записываем данные о весах в файл
model.save_weights(directory + "/mnist_model.h5")